# FINAL NOTEBOOK

In [1]:
%matplotlib inline
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from scipy.stats import uniform, randint
from xgboost.sklearn import XGBClassifier 
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In /home/paul/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/paul/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/paul/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


#### Loading of the data

In [2]:
X = np.loadtxt('xtrain_challenge.csv', delimiter=',', skiprows = 1)
Y = np.loadtxt('ytrain_challenge.csv', delimiter=',', skiprows = 1)

In [3]:
#Chargement des données test
X_exam = np.loadtxt('xtest_challenge.csv', delimiter=',', skiprows = 1)

#### We shuffle the data

In [7]:

""""X1=X[Y==1]
X0=X[Y==0]
Y1=Y[Y==1]
Y0=Y[Y==0]"""

N=X.shape[0]
ordre=np.arange(0,N)
np.random.shuffle(ordre)

# Use as new variables
X=X[ordre[:]]
Y=Y[ordre[:]]

#X=np.concatenate((X0,X1))
#Y=np.concatenate((Y0,Y1))

#### Observation of the data

In [27]:
Y.shape

(1068504,)

In [28]:
X.shape

(1068504, 37)

In [29]:
X_exam.shape

(3318296, 37)

In [30]:
Y.mean()

0.03705461093266848

We observe that 96% of the data has the label has the label 1. 

#### Scale of the data

XXXXXXX

#### Division of the data in training set/ test set

Sometimes, we are going to use only 1/8 of our data, to make our training faster

In [8]:
N=len(X)

X=X[:N//8]
Y=Y[:N//8]

In [9]:
X_train, X_test ,y_train, y_test = train_test_split(X,Y, test_size=0.33, random_state=42
)

## First try : KNN

In [33]:
KNN = KNeighborsClassifier(n_neighbors=10)
KNN.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [35]:
KNN.score(X_train,y_train)

0.9974968431168774

In [36]:
NN.score(X_test,y_test)

0.9970732371358563

Dataset| Scores 
- |-: 
training set  | 99,75%
test set | 99,71% 

We see that even we a very simple model, we reach excellent scores.

Without looking as the best parameter n_neihbors, we are going to test directly a more complex classifier.

## SVM

In [10]:
mean_on_train=X_train.mean(axis=0)
std_on_train=X_train.std(axis=0)

X_train_scaled=(X_train-mean_on_train)/std_on_train
X_test_scaled=(X_test-mean_on_train)/std_on_train
X_exam_scaled=(X_exam-mean_on_train)/std_on_train

In [108]:
svm1=LinearSVC(C=0.1,class_weight={0:1,1:1})
svm1.fit(X_train_scaled,y_train)

/home/paul/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=0.1, class_weight={0: 1, 1: 1}, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [109]:
svm1.score(X_train_scaled,y_train)

0.997932660609921

In [110]:
svm1.score(X_test_scaled,y_test)

0.9979580724203648

We are looking for best parameters

In [95]:
parameters={'C':[0.01,0.1,1],
           'class_weight': [{0:1,1:1},{0:1,1:5},{0:1,1:15}]}

In [96]:
grid = GridSearchCV(LinearSVC(), parameters)
grid.fit(X_train_scaled,y_train)

/home/paul/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/paul/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/paul/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/paul/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/paul/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the numb

GridSearchCV(cv=None, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1],
                         'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 5},
                                          {0: 1, 1: 15}]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [97]:
grid.best_params_

{'C': 0.1, 'class_weight': {0: 1, 1: 1}}

In [98]:
grid.best_score_

0.9977985661648525

In [111]:
svm2=LinearSVC(C=0.01,class_weight={0:1,1:1})

In [112]:
svm2.fit(X_train_scaled,y_train)

LinearSVC(C=0.01, class_weight={0: 1, 1: 1}, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [113]:
svm2.score(X_train_scaled,y_train)

0.9978656117648373

In [114]:
svm2.score(X_test_scaled,y_test)

0.9978673200834921

Dataset| Scores 
- |-: 
training set  | 99,787%
test set | 99,787% 

## MLP Classifier

In [11]:
mlp=MLPClassifier(hidden_layer_sizes=(15,),verbose=1,max_iter=500,random_state=0)

In [12]:
mlp.fit(X_train_scaled,y_train)

Iteration 1, loss = 0.15308718
Iteration 2, loss = 0.02765097
Iteration 3, loss = 0.01723279
Iteration 4, loss = 0.01312226
Iteration 5, loss = 0.01078041
Iteration 6, loss = 0.00936646
Iteration 7, loss = 0.00841384
Iteration 8, loss = 0.00766408
Iteration 9, loss = 0.00714035
Iteration 10, loss = 0.00676605
Iteration 11, loss = 0.00645041
Iteration 12, loss = 0.00627225
Iteration 13, loss = 0.00610646
Iteration 14, loss = 0.00601082
Iteration 15, loss = 0.00580699
Iteration 16, loss = 0.00574238
Iteration 17, loss = 0.00558155
Iteration 18, loss = 0.00553334
Iteration 19, loss = 0.00543309
Iteration 20, loss = 0.00533248
Iteration 21, loss = 0.00529160
Iteration 22, loss = 0.00514960
Iteration 23, loss = 0.00511403
Iteration 24, loss = 0.00503070
Iteration 25, loss = 0.00500439
Iteration 26, loss = 0.00488087
Iteration 27, loss = 0.00484650
Iteration 28, loss = 0.00482451
Iteration 29, loss = 0.00473317
Iteration 30, loss = 0.00468218
Iteration 31, loss = 0.00461855
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=False)

In [13]:
mlp.score(X_train_scaled,y_train)

0.9988489948260642

In [14]:
mlp.score(X_test_scaled,y_test)

0.9982757055994191

Dataset| Scores 
- |-: 
training set  | 99,88%
test set | 99,82% 

Without doing any gridsearch, just by testing a MLP with one layer and 15 neurons, we reach very good scores.
We are going to test if we can improve it by doing a grisearch on certain parameters of our MLP.

In [125]:
res=[]
for i in range(20):
    for j in range(20):
        res.append((i,j))
    res.append((i,))

param = {
    'hidden_layer_sizes': res}

model = MLPClassifier(max_iter=300,verbose=1)
grid = RandomizedSearchCV(estimator = model, param, n_iter=100, verbose=1, cv=3)
grid.fit(X_train_scaled, y_train)

print('The best hidden_layer_sizes is', grid.best_params_.get('hidden_layer_sizes'), ' with an average validation score equal to ', grid.best_score_)


# Let's now use the best model to assess the test score
BestModel=grid.best_estimator_

print('The test score is', BestModel.score(X_test_scaled, y_test))

SyntaxError: positional argument follows keyword argument (<ipython-input-125-8e535e71286c>, line 11)

## XGBOOST

In an attempt to further improve our score, we decided to learn more about the models that frequently win machine learning competitions. 
That's how we found a model that frequently seemed to be above the others, called xgboost. 

What is XGBoost? 

XGBoost is a decision-tree-based ensemble Machine Learning algorithm that uses a gradient boosting framework.

In [10]:
params_fixed={'objective':'binary:logistic','silent':0}

In [11]:
params_dist_grid={
    'max_depth':range(3,10,2),
    'gamma':[0,0.5,1],
    'n_estimators':randint(30,500),
    'learning_rate':uniform(),
    'subsample':uniform(),
    'colsample_bytree':uniform()
}

Ne pas toucher c'est très long

In [23]:
rd_grid=RandomizedSearchCV(
    estimator=XGBClassifier(**params_fixed,seed=seed),param_distributions=params_dist_grid,
n_iter=50,
cv=3,
scoring='accuracy',
random_state=seed,
verbose=10
)

In [24]:
rd_grid.fit(X_train,y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605, score=0.997, total=   4.3s
[CV] colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


[CV]  colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605, score=0.996, total=   4.2s
[CV] colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605 


...

[CV]  colsample_bytree=0.7494207571997938, gamma=1, learning_rate=0.5669082329780527, max_depth=3, n_estimators=35, subsample=0.8936616121288703, score=0.998, total=   6.6s
[CV] colsample_bytree=0.7494207571997938, gamma=1, learning_rate=0.5669082329780527, max_depth=3, n_estimators=35, subsample=0.8936616121288703 
[CV]  colsample_bytree=0.7494207571997938, gamma=1, learning_rate=0.5669082329780527, max_depth=3, n_estimators=35, subsample=0.8936616121288703, score=0.998, total=   6.7s
[CV] colsample_bytree=0.7494207571997938, gamma=1, learning_rate=0.5669082329780527, max_depth=3, n_estimators=35, subsample=0.8936616121288703 
[CV]  colsample_bytree=0.7494207571997938, gamma=1, learning_rate=0.5669082329780527, max_depth=3, n_estimators=35, subsample=0.8936616121288703, score=0.999, total=   6.6s
[CV] colsample_bytree=0.1233781606164811, gamma=0.5, learning_rate=0.09772802614862519, max_depth=3, n_estimators=295, subsample=0.32162421002224917 
[CV]  colsample_bytree=0.1233781606164811

[CV]  colsample_bytree=0.45294600144107167, gamma=1, learning_rate=0.08303834961595746, max_depth=7, n_estimators=187, subsample=0.6069127728741993, score=0.999, total=  51.9s
[CV] colsample_bytree=0.7327546305008223, gamma=0.5, learning_rate=0.692803142146249, max_depth=7, n_estimators=180, subsample=0.705851693379962 
[CV]  colsample_bytree=0.7327546305008223, gamma=0.5, learning_rate=0.692803142146249, max_depth=7, n_estimators=180, subsample=0.705851693379962, score=0.998, total=66.2min
[CV] colsample_bytree=0.7327546305008223, gamma=0.5, learning_rate=0.692803142146249, max_depth=7, n_estimators=180, subsample=0.705851693379962 
[CV]  colsample_bytree=0.7327546305008223, gamma=0.5, learning_rate=0.692803142146249, max_depth=7, n_estimators=180, subsample=0.705851693379962, score=0.999, total= 1.1min
[CV] colsample_bytree=0.7327546305008223, gamma=0.5, learning_rate=0.692803142146249, max_depth=7, n_estimators=180, subsample=0.705851693379962 
[CV]  colsample_bytree=0.7327546305008

[CV]  colsample_bytree=0.919421811544759, gamma=1, learning_rate=0.9622594737743404, max_depth=5, n_estimators=158, subsample=0.6372091818441632, score=0.998, total=  55.1s
[CV] colsample_bytree=0.919421811544759, gamma=1, learning_rate=0.9622594737743404, max_depth=5, n_estimators=158, subsample=0.6372091818441632 
[CV]  colsample_bytree=0.919421811544759, gamma=1, learning_rate=0.9622594737743404, max_depth=5, n_estimators=158, subsample=0.6372091818441632, score=0.998, total=  55.6s
[CV] colsample_bytree=0.6754086694441321, gamma=0, learning_rate=0.22800333163252395, max_depth=9, n_estimators=165, subsample=0.4357666773827713 
[CV]  colsample_bytree=0.6754086694441321, gamma=0, learning_rate=0.22800333163252395, max_depth=9, n_estimators=165, subsample=0.4357666773827713, score=0.999, total= 1.0min
[CV] colsample_bytree=0.6754086694441321, gamma=0, learning_rate=0.22800333163252395, max_depth=9, n_estimators=165, subsample=0.4357666773827713 
[CV]  colsample_bytree=0.675408669444132

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 199.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f049e0eccf8>,
                                        'max_depth'

In [25]:
rd_grid.best_score_

0.9988126784811241

In [26]:
rd_grid.best_params_

{'colsample_bytree': 0.5213285655977811,
 'gamma': 1,
 'learning_rate': 0.12245745743134928,
 'max_depth': 5,
 'n_estimators': 448,
 'subsample': 0.48831719352338554}

In [6]:
clf=XGBClassifier(colsample_bytree= 0.5213285655977811,
 gamma= 1,
 learning_rate= 0.12245745743134928,
 max_depth= 5,
 n_estimators= 448,
 subsample= 0.48831719352338554,objective='binary:logistic')

In [8]:
clf.fit(X,Y)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5213285655977811, gamma=1,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.12245745743134928, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=448, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.48831719352338554,
              tree_method=None, validate_parameters=False, verbosity=None)

In [8]:
clf.score(X_test,y_test)

0.9988797726647514

Test 2 

In [14]:
rd_grid=RandomizedSearchCV(
    estimator=XGBClassifier(**params_fixed,seed=seed),param_distributions=params_dist_grid,
n_iter=100,
cv=3,
scoring='accuracy',
random_state=seed,
verbose=10
)

In [15]:
rd_grid.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605, score=0.997, total=   3.3s
[CV] colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV]  colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605, score=0.997, total=   3.3s
[CV] colsample_bytree=0.06503439684192913, gamma=0, learning_rate=0.24756150723102166, max_depth=7, n_estimators=34, subsample=0.11848249237448605 


...

[CV]  colsample_bytree=0.4715416412473765, gamma=1, learning_rate=0.2662630192036526, max_depth=5, n_estimators=369, subsample=0.21027234582906684, score=0.998, total= 1.0min
[CV] colsample_bytree=0.4715416412473765, gamma=1, learning_rate=0.2662630192036526, max_depth=5, n_estimators=369, subsample=0.21027234582906684 
[CV]  colsample_bytree=0.4715416412473765, gamma=1, learning_rate=0.2662630192036526, max_depth=5, n_estimators=369, subsample=0.21027234582906684, score=0.999, total= 1.0min
[CV] colsample_bytree=0.4715416412473765, gamma=1, learning_rate=0.2662630192036526, max_depth=5, n_estimators=369, subsample=0.21027234582906684 
[CV]  colsample_bytree=0.4715416412473765, gamma=1, learning_rate=0.2662630192036526, max_depth=5, n_estimators=369, subsample=0.21027234582906684, score=0.999, total= 1.0min
[CV] colsample_bytree=0.5139404691578969, gamma=0.5, learning_rate=0.2999675136645916, max_depth=9, n_estimators=278, subsample=0.8277418366554766 
[CV]  colsample_bytree=0.51394046

[CV]  colsample_bytree=0.9564638424139361, gamma=0, learning_rate=0.28181496645949267, max_depth=7, n_estimators=425, subsample=0.3361040265850367, score=0.999, total= 2.4min
[CV] colsample_bytree=0.3625473603983238, gamma=0.5, learning_rate=0.886975127977201, max_depth=3, n_estimators=420, subsample=0.04016312842699654 
[CV]  colsample_bytree=0.3625473603983238, gamma=0.5, learning_rate=0.886975127977201, max_depth=3, n_estimators=420, subsample=0.04016312842699654, score=0.997, total=  32.4s
[CV] colsample_bytree=0.3625473603983238, gamma=0.5, learning_rate=0.886975127977201, max_depth=3, n_estimators=420, subsample=0.04016312842699654 
[CV]  colsample_bytree=0.3625473603983238, gamma=0.5, learning_rate=0.886975127977201, max_depth=3, n_estimators=420, subsample=0.04016312842699654, score=0.997, total=  32.0s
[CV] colsample_bytree=0.3625473603983238, gamma=0.5, learning_rate=0.886975127977201, max_depth=3, n_estimators=420, subsample=0.04016312842699654 
[CV]  colsample_bytree=0.3625

[CV]  colsample_bytree=0.5060506125373518, gamma=0.5, learning_rate=0.64099934659491, max_depth=9, n_estimators=166, subsample=0.3027654384906646, score=0.998, total=  42.4s
[CV] colsample_bytree=0.5060506125373518, gamma=0.5, learning_rate=0.64099934659491, max_depth=9, n_estimators=166, subsample=0.3027654384906646 
[CV]  colsample_bytree=0.5060506125373518, gamma=0.5, learning_rate=0.64099934659491, max_depth=9, n_estimators=166, subsample=0.3027654384906646, score=0.998, total=  43.7s
[CV] colsample_bytree=0.9983862721241882, gamma=0.5, learning_rate=0.2159654911802783, max_depth=5, n_estimators=232, subsample=0.9385444349357669 
[CV]  colsample_bytree=0.9983862721241882, gamma=0.5, learning_rate=0.2159654911802783, max_depth=5, n_estimators=232, subsample=0.9385444349357669, score=0.999, total= 1.4min
[CV] colsample_bytree=0.9983862721241882, gamma=0.5, learning_rate=0.2159654911802783, max_depth=5, n_estimators=232, subsample=0.9385444349357669 
[CV]  colsample_bytree=0.998386272

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 257.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc68131c710>,
                                        'max_depth'

In [19]:
rd_grid.best_params_

{'colsample_bytree': 0.9983862721241882,
 'gamma': 0.5,
 'learning_rate': 0.2159654911802783,
 'max_depth': 5,
 'n_estimators': 232,
 'subsample': 0.9385444349357669}

In [20]:
clf=XGBClassifier(colsample_bytree= 0.9983862721241882,
 gamma= 0.5,
 learning_rate= 0.2159654911802783,
 max_depth= 5,
 n_estimators= 232,
 subsample= 0.9385444349357669,objective='binary:logistic')

Pour l'instant le meilleur avec X 

In [21]:
clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9983862721241882,
              gamma=0.5, gpu_id=-1, importance_type='gain',
              interaction_constraints=None, learning_rate=0.2159654911802783,
              max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=232, n_jobs=0,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.9385444349357669, tree_method=None,
              validate_parameters=False, verbosity=None)

In [22]:
clf.score(X_test,y_test)

0.9989450010918671

In [6]:
clf = xgb.XGBClassifier(missing=np.nan, max_depth=9, 
                        n_estimators=300, learning_rate=0.01,gamma=0.1, min_child_weight=5,
                        subsample=1, colsample_bytree=0.9, seed=2100,objective= 'binary:logistic')

In [7]:
clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=9,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=2100, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2100, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [8]:
clf.score(X_test,y_test)

0.9984798940463462

In [ ]:
ytest1=clf.predict(X_exam[:1500000])


0,9984 with max_depth=11 
0,9984 with max_depth=9
0,9980 with max_depth=5
0,9983 with max_depth=7
0,9975 with max_depth=3

### Tune max_depth and min_child_weight

In [110]:
param_test1 = {
 'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearchCV(estimator = XGBClassifier( max_depth=9, learning_rate =0.1, n_estimators=200, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:', nthread=4, scale_pos_weight=1, seed=27),
                        
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)


gsearch1.fit(X_train,y_train)

print("Best score", gsearch1.best_score_)
print("Best param ", gsearch1.best_params_)

OSError: [Errno 12] Cannot allocate memory

In [21]:
param_test2 = {
 'max_depth':range(9,14,2),
 'min_child_weight':range(4,7)
}

gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=50, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
                        
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)


gsearch2.fit(X_train,y_train)

print("Best score", gsearch2.best_score_)
print("Best param ", gsearch2.best_params_)

/home/paul/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Best score 0.9996823084678426
Best param  {'max_depth': 9, 'min_child_weight': 5}


We keep {'max_depth': 11, 'min_child_weight': 5}

### We tune gamma

In [75]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=70, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                        
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch3.fit(X_train,  y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/home/paul/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [76]:
gsearch3.best_params_

{'gamma': 0.2}

In [77]:
gsearch3.best_score_

0.9994678530318669

### Now, let's test this model

In [22]:
model=XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=9,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

In [23]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)

In [24]:
model.score(X_test,y_test)

0.9983044914568246

In [9]:
ytest = clf.predict(X_exam)
print(ytest.mean())
np.savetxt('ytest_challenge_student2.csv', ytest, fmt = '%1.0d', delimiter=',')

0.03488447082478477


In [20]:
uniform()